# Falling Object (1D)
Variational Integration using Discrete Euler Lagrange formulation

Following http://ddg.cs.columbia.edu/SIGGRAPH06/stern-siggraph-talk.pdf

In [1]:
using RigidBodyDynamics
using StaticArrays
using SymPy
using Plots; plotly();

In [2]:
masses = @syms m positive = true
gravitational_acceleration = @syms g real = true
params = [masses..., gravitational_acceleration...]
transpose(params)

1×2 LinearAlgebra.Transpose{Sym,Array{Sym,1}}:
 m  g

In [3]:
T = Sym # the 'scalar type' of the Mechanism we'll construct
axis = SVector(zero(T), zero(T), one(T)) # axis of rotation for each of the joints
freefall = Mechanism(RigidBody{T}("world"); gravity = SVector(zero(T), zero(T), -g))
world = root_body(freefall) # the fixed 'world' rigid body

RigidBody: "world"

In [4]:
inertia = SpatialInertia(CartesianFrame3D("block"),
    moment=0. * axis * transpose(axis),
    com=SVector(zero(T), zero(T), zero(T)),
    mass=m)
body = RigidBody(inertia)
joint = Joint("shoulder", Prismatic(axis))
joint_to_world = one(Transform3D{T}, frame_before(joint), default_frame(world));
attach!(freefall, world, body, joint,
    joint_pose = joint_to_world);

In [5]:
x = MechanismState(freefall);
simplify.(transpose([x.q..., x.v...]))

1×2 Array{Sym,2}:
 0  0

In [6]:
q = configuration(x)
v = velocity(x)
q[1], v[1] = symbols("q", real = true), symbols("v", real = true)

(q, v)

In [18]:
@show x

x = MechanismState{Sym, Sym, Sym, …}(…)


MechanismState{Sym, Sym, Sym, …}(…)

In [7]:
simplify.(mass_matrix(x))

1×1 Array{Sym,2}:
 m

In [8]:
simplify(kinetic_energy(x))

   2
m*v 
----
 2  

In [9]:
simplify(gravitational_potential_energy(x))

g*m*q

## Compute the continuous Lagrangian
$$
L(q, \dot{q}) = K(\dot{q}) - V(q)
$$

In [10]:
function lagrangian(q, v)
    y = MechanismState(freefall, [q...], [v...])
    return kinetic_energy(y) - gravitational_potential_energy(y)
end
L = lagrangian(q, v)
simplify(L)

  /          2\
m*\-2*g*q + v /
---------------
       2       

Compute discrete Lagrangian using midpoint quadrature
$$
\begin{align}
L^d(q_k, q_{k+1}) &\approx \int_{t_k}^{t_{k+1}} L(q, \dot{q}) dt \\
&= h L\left( \frac{q_k + q_{k+1}}{2}, \frac{q_{k+1} - q_k}{h} \right)
\end{align}
$$

In [11]:
h = symbols("h", positive = true)
function discrete_lagrangian(q_k1, q_k2)    
    h * lagrangian((q_k1 + q_k2) / 2, (q_k2 - q_k1) / h)
end;

## Discrete Euler-Lagrange Equations
Following discrete action principle $\delta S^d(q) = 0$ where
$$
S^d(q) = \sum_{k=0}^{N-1} L^d (q_k, q_{k+1}),
$$
the discrete Euler-Lagrange Equations must satisfy
$$
D_1 L^d(q_k, q_{k+1}) + D_2 L^d(q_{k-1}, q_{k}) = 0
$$

In [12]:
N = length(q)
z_k = SVector{N}([symbols("z$(i)_k", real = true) for i ∈ 1:N])
z_kp = SVector{N}([symbols("z$(i)_k-1", real = true) for i ∈ 1:N])
z_kn = SVector{N}([symbols("z$(i)_k+1", real = true) for i ∈ 1:N])
L1 = discrete_lagrangian(z_k, z_kn)
L2 = discrete_lagrangian(z_kp, z_k)
dL1 = SVector{N}([diff(L1, z_k[i]) for i ∈ 1:N])
dL2 = SVector{N}([diff(L2, z_k[i]) for i ∈ 1:N])
dL = dL1 + dL2
simplify.(dL)[1]

  /     2                           \
m*\- g*h  + 2*z1_k - z1_k+1 - z1_k-1/
-------------------------------------
                  h                  

Solve DEL for $q_{k+1}$. Note that the mass gets cancelled out.

In [13]:
var_int = SVector{N}([solve(dL[i], z_kn[i]) for i ∈ 1:N])
var_int[1]

1-element Array{Sym,1}:
 -g*h^2 + 2*z1_k - z1_k-1

*Lambdify* symbolic expression for variational integrator to make it executable

In [14]:
integrator = lambdify(var_int[1])
free_symbols(var_int[1])

4-element Array{Sym,1}:
      g
      h
   z1_k
 z1_k-1

## Example visualization
Variational integration of discrete mechanics vs. Euler integration.
For the variational integrator, we need to provide two positions $q_{k-1}, q_k$.
In our freefalling object scenario, the distance travelled at the first time step $\Delta t$ is $\frac{1}{2}g \Delta t^2$.

In [15]:
gravity = 9.81
Δt = 0.1  # time step
times = 0:Δt:10

start_pos = 10.
pos_vi = [start_pos, start_pos - 0.5 * gravity * Δt^2]
pos_euler = [start_pos]  # store position and velocity
vel_euler = 0.
for (k, t) ∈ enumerate(times[1:end-1])
    vel_euler += -gravity * Δt
    push!(pos_euler, pos_euler[k] + vel_euler * Δt)
    k < 2 && continue
    push!(pos_vi, integrator(gravity, Δt, pos_vi[k], pos_vi[k-1])[1])
end

plot(times, pos_vi, label="VI")
plot!(times, [p[1] for p in pos_euler], label="Euler")

<!DOCTYPE html>
 
 
 Plots.jl

### Integration at a lower resolution
The discrete mechanics formulation enables variational integrators to maintain time resolution independence.

In [16]:
Δt_low = 2  # time step
times_low = 0:Δt_low:10

pos_vi_low = [start_pos, start_pos - 0.5 * gravity * Δt_low^2]
pos_euler_low = [start_pos]  # store position and velocity
vel_euler = 0.
for (k, t) ∈ enumerate(times_low[1:end-1])
    vel_euler += -gravity * Δt_low
    push!(pos_euler_low, pos_euler_low[k] + vel_euler * Δt_low)
    k < 2 && continue
    push!(pos_vi_low, integrator(gravity, Δt_low, pos_vi_low[k], pos_vi_low[k-1])[1])
end

plot(times, pos_vi, label="VI before")
plot!(times_low, pos_vi_low, seriestype=:path, label="VI")
plot!(times_low, [p[1] for p in pos_euler_low], label="Euler")

<!DOCTYPE html>
 
 
 Plots.jl